# Ejercicio: Implementar un método callbacks usando el conjuntod e datos MNIST

Hemos aprendido a clasificar usando Fashion MNIST, un conjunto de datos que contiene prendas de vestir. Hay otro conjunto de datos similar llamado MNIST que tiene elementos de escritura a mano: los dígitos del 0 al 9.

Escriba un clasificador MNIST con una precisión del 99 % o más, y lo haga sin un número fijo de épocas; es decir, debe dejar de entrenar una vez que alcance ese nivel de precisión.

Algunas notas:
1. Dada la arquitectura de la red, deberemos tener éxito en menos de 10 épocas.
2. Cuando alcance el 99 % o más, debes imprimir la cadena "Alcanzó el 99 % de precisión, por lo que entrenamiento cancelado". y dejar de entrenar.

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Comenzamos cargando los datos pero debemos tener un par de cosas a tener en cuenta:

- El archivo `mnist.npz` lo debemos descargar y subir a nuestra unidad de Google Drive, a partir de aquí debemos indicar la ruta en la que se encuentra en el método ya implementado en la API de Keras.

- `load_data` devuelve los conjuntos de entrenamiento y test en forma de tuplas `(x_train, y_train), (x_test, y_test)` pero en este ejercicio solo necesitaramos el conjunto de entrenamiento por lo que podemos ignorar la segunda tupla.

In [ ]:
# Obtenemos los datos y descartamos los test
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data('/content/drive/MyDrive/PIA23-24/tema3/lib5/mnist.npz')

# Normalizamos los datos
X_train = X_train / 255.0
X_test = X_test /255.0

Echa un vistazo al conjunto de datos

In [10]:
data_estructura = X_train.shape

print(f"Existen {data_estructura[0]} ejemplos de las siguientes dimensiones ({data_estructura[1]}, {data_estructura[2]})")
X_train

Existen 60000 ejemplos de las siguientes dimensiones (28, 28)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

Ahora es el momento de crear nuestro propia clase de callbacks.

In [ ]:
# Recuerda heredar de la clase correspondiente
class myCallback(tf.keras.callbacks.Callback):
        # Define la función correcta para on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
          if(logs.get('loss')<=0.1):
            print("La perdida ha bajado del 30%, se cancela el entrenamiento")
            self.model.stop_training = True

Ahora que has definido tu clase callbacks desarrolla `entrenar_mnist` para entrenar el modelo

In [11]:
def entrenar_mnist(x_entrenamiento, y_entrenamiento):


    # Instancia la classe callback
    callbacks = myCallback()

    # Define el modelo con al menos 3 capas
    model = tf.keras.models.Sequential([
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ])

    # Compila el modelo
    model.compile(loss="sparse_categorical_crossentropy", optimizer = "adam", metrics=['accuracy'])

    # Entrena el modelo
    historico = model.fit(x_entrenamiento, y_entrenamiento, epochs=12, callbacks=[callbacks])

    return model

Llama al método pasándole el conjunto de datos

In [ ]:
carlos = entrenar_mnist(X_train, y_train)

Epoch 1/12
1875/1875 [==============================] - 9s 4ms/step - loss: 0.2568 - accuracy: 0.9279
Epoch 2/12
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1119 - accuracy: 0.9666
Epoch 3/12
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0738 - accuracy: 0.9776


Si ve el mensaje "Alcanzó el 99% de precisiónn por

---

lo que entrenamiento cancelado" impreso después de menos de 10 épocas es una señal de buen trabajo.

In [ ]:
carlos.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0849 - accuracy: 0.9744


[0.08491947501897812, 0.974399983882904]